PYTHON NOTEBOOK USED TO ANSWER TO EXERCISES OF CHAPTER 3 OF MATH80624 LECTURE NOTES

Modified by:
1. Chun Peng (Created for RSOME January 2021)
2. Erick Delage (January 2021)

As discussed in Chapter 3 of the  [lecture notes](http://web.hec.ca/pages/erick.delage/MATH80624_LectureNotes.pdf) of MATH80624 at HEC Montréal. 

WARNING!!!

The following code exploits a free Mosek licence for the course MATH80624 at HEC Montréal (expiration June 1st 2021). If you have error messages informing you about licencing issues, you may try uncommenting the installation lines for Gurobi. Otherwise, we recommend that you obtain your own licence of either Mosek ([url](https://www.mosek.com/)) or Gurobi ([url](https://www.gurobi.com/)).

**Jean-Sébastien Matte**

**Sena Onen Oz**

# **Preliminaries**

In [ ]:
!pip install rsome
!pip install mosek
!rm mosek.lic
!git clone https://github.com/erickdelage/80624
!cp ./80624/mosek.lic .
!cp ./80624/stockData.mat .
!rm -r ./80624
!mkdir -p /root/mosek
!cp ./mosek.lic /root/mosek
#!pip install -i https://pypi.gurobi.com gurobipy

     |████████████████████████████████| 10.1MB 5.2MB/s 
rm: cannot remove 'mosek.lic': No such file or directory
Cloning into '80624'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 2), reused 11 (delta 1), pack-reused 0
Unpacking objects: 100% (12/12), done.


In [ ]:
import rsome as rso
import numpy as np
from rsome import ro
from rsome import msk_solver as my_solver  #Import Mosek solver interface
#from rsome import grb_solver as my_solver  #Import Gurobi solver interface
from scipy.io import loadmat         # load the mat file

### Load the data

In [ ]:
x = loadmat('stockData.mat')  # load the mat file
Rs=x['stockData'][0][0]['returns'] #Rs contains the data used for calibration (2000-2009)
RsTest=x['stockDataTest'][0][0]['returns'] #Rs contains the data used for calibration (2010-2014)
(nStocks,nMonths)=Rs.shape

### Define Value-at-Risk level

In [ ]:
VaRProb = 0.95 
VaReps = 1-VaRProb

### Manipulate the calibration data to reach the form $ r = \mu+P Z\;.$

In this exercise, we reformulate the source of uncertainty in terms of a vector of uncertain z's that lie in the $[-1, 1]^m$ box.

In [ ]:
mu=np.mean(Rs, axis=1)
tmp = Rs - mu.reshape(-1,1)@np.ones((1,nMonths))
tmp_max = abs(tmp).max(1)
Zs = np.diag(1/tmp_max)@tmp
P = np.diag(tmp_max)

### Define a few useful functions

The "getWCdist" function returns a distribution that spreads the weight on all extreme points of the box support [-1, 1]^m

In [ ]:
def getWCdist(m):
    # Returns a uniform distribution over the extreme points of [-1, 1]^m
    # When calling [vals,ps] = getWCdist(m)
    #    m: size of the random vector
    #    vals : m x N matrixarray of values that the distribution takes
    #    ps : array of probability associated to each value

    if (m==1):
        vals=np.array([-1, 1]).reshape(1,2)
        ps = np.array([0.5, 0.5])
    else:
        (vals0,ps0)=getWCdist(m-1)
        n=np.size(vals0,axis=1)
        v=np.concatenate((-1*np.ones(n), np.ones(n)))
        val=np.concatenate((vals0,vals0),axis=1)
        vals=np.concatenate(([v],val),axis=0)   
        ps = np.concatenate((0.5*ps0, 0.5*ps0))
    return (vals, ps)

#getWCdist allows us to create a set of worst-case scenarios for returns
tmp=P@getWCdist(nStocks)[0]
WCRETURNS = mu.reshape(-1,1)@np.ones((1,tmp.shape[1]))+tmp

def testPolicyVaR(x,returns,VaReps):
    tmp = sorted(x@returns)
    VaR=-tmp[int(np.floor(VaReps*returns.shape[1]))]
    return VaR


# **Exercise 3.1) Calibration of uncertainty sets using data**

For each of the three uncertainty sets below, calibrate the size parameter in order for $\mathcal{Z}$ to include 95\% of the observed realization:

Budgeted uncertainty set, i.e. $\mathcal{Z}:= \{z\in\mathbb{R}^m\,|\,z_i\in[-1,\,1],, \|z\|_1 \leq \Gamma\}$

Boxed ellipsoidal set, i.e. $\mathcal{Z}:= \{z\in\mathbb{R}^m\,|\,z_i\in[-1,\,1],, \|z\|_2 \leq \gamma\}$

CVaR uncertainty set, i.e. 
$$\mathcal{Z}:= \left\{ z\in \mathbb{R}^m\,\middle|\, \exists \theta\in\mathbb{R}^K,\, z = \sum_{i=1}^K \theta_i \bar{z}_i,\, \theta\geq 0, \, \sum_{i=1}^K \theta_i = 1,\, \theta\leq \frac{1}{K\alpha}\right\}.$$

Note that we already provide below the calibration scheme for the ellipsoidal set, hence you are only asked to calibrate the budgeted and CVaR sets. 

Note finally that when using the CVaR uncertainty set, you can consider the $\alpha$ to be the size parameter and simply let the $\bar{z}_i$ to be the set of all observed realizations. Also, if one wishes to find the largest $\alpha$ such that $z\in\mathcal{Z}$, then he simply needs to solve the following linear program:
\begin{eqnarray*}
\min_{\theta,\gamma}\;&& \gamma\\
\text{s.t.} && z = \sum_{i=1}^K\theta_i \bar{z}_i\\
&&0 \leq \theta \leq \gamma/K\\
&& \sum_{i=1}^K \theta_i= 1\,,
\end{eqnarray*}
and select $\alpha = 1/\gamma$.

### Calibrating the boxed ellipsoidal uncertainty set

In [ ]:
# Calibrating the ellipsoid set
tmp=np.sort(np.linalg.norm(Zs,axis=0))
gamma=tmp[int(np.ceil((1-VaReps)*len(tmp)))-1]
print('Calibrating the ellipsoid set: gamma={0:0.6f}'.format(gamma))

Calibrating the ellipsoid set: gamma=1.457444


### Calibrating the budgeted uncertainty set

In [ ]:
# Calibrating the budgeted set
tmp=np.sort(np.linalg.norm(Zs, ord=1,axis=0))
Gamma=tmp[int(np.ceil((1-VaReps)*len(tmp)))-1]
print('Calibrating the budgeted set: Gamma={0:0.6f}'.format(Gamma))

Calibrating the budgeted set: Gamma=3.606350


### Calibrating the CVaR uncertainty set


In [ ]:
model_CVaR=ro.Model('CVaR_Calibration')  
# Define decision variable(s)
lmd = model_CVaR.dvar(1)
t = model_CVaR.dvar(nMonths)
z = model_CVaR.dvar(nStocks)

model_CVaR.min(lmd)

model_CVaR.st(z==(Zs@t))
model_CVaR.st(t>=0)
model_CVaR.st((nMonths*t)<=lmd)
model_CVaR.st(sum(t) == 1)

model_CVaR.solve(my_solver)

print('The lambda is {0:0.4f}'.format(model_CVaR.get()))
print('The alpha is {0:0.4f}'.format(1/lmd.get()[0]))

Being solved by Mosek...
Solution status: optimal
Running time: 0.0113s
The lambda is 1.0000
The alpha is 1.0000


# **Exercise 3.2) Calibration of uncertainty sets using distribution hypothesis**

For each of the two uncertainty sets below, calibrate the size parameter in order for $\mathcal{Z}$ to be such that a robust linear constraint employing $\mathcal{Z}$ is guaranteed to return a solution that will satisfy the chance constraint with 95\% probability as long as the distribution of $Z$ satisfies assumption 3.4:

Budgeted uncertainty set, i.e. $\mathcal{Z}:= \{z\in\mathbb{R}^m\,|\,z_i\in[-1,\,1],, \|z\|_1 \leq \Gamma\}$

Boxed ellipsoidal set, i.e. $\mathcal{Z}:= \{z\in\mathbb{R}^m\,|\,z_i\in[-1,\,1],, \|z\|_2 \leq \gamma\}$.

### Calibrating the budgeted uncertainty set

In [ ]:
# From Corollary 3.8, we have
Gamma2 = np.sqrt(2 * Zs.shape[0] * np.log(1 / VaReps))
print('Calibrating the budgeted set: Gamma={0:0.6f}'.format(Gamma2))

Calibrating the budgeted set: Gamma=7.740455


### Calibrating the ellipsoid set

In [ ]:
# From Theorem 3.5, we have
gamma2 = np.sqrt(2 * np.log(1 / VaReps))
print('Calibrating the ellipsoid set: gamma={0:0.6f}'.format(gamma2))

Calibrating the ellipsoid set: gamma=2.447747


# **Exercise 3.3) Evaluate the portfolios obtained by each method**

### Boxed ellipsoidal set from Exercise 3.1)
$$\mathcal{Z}:= \{z\in\mathbb{R}^m\,|\,z_i\in[-1,\,1],, \|z\|_2 \leq \gamma\}$$

In [ ]:
model = ro.Model('solveRobustPortfolio_Ellipsoidal')
x=model.dvar(nStocks)

z=model.rvar(nStocks)

EllipsoidSet=(z>=-1, z<=1, rso.norm(z,2)<=gamma)
model.minmax(-(mu+P@z)@x,EllipsoidSet)
model.st(x<=1)
model.st(sum(x)==1)
model.st(x>=0)
model.solve(my_solver)

obj_Ellipsoid=model.get()
xx_Ellipsoid=x.get() #get optimal portfolio

print('Estimated VaR is {0:0.4f}'.format(obj_Ellipsoid))
print('VaR from 2000 to 2009 is {0:0.4f}'.format(testPolicyVaR(xx_Ellipsoid, Rs,VaReps)))
print('VaR from 2010 to 2014 is {0:0.4f}'.format(testPolicyVaR(xx_Ellipsoid, RsTest,VaReps)))
print('VaR with extreme distribution is {0:0.4f}'.format(testPolicyVaR(xx_Ellipsoid, WCRETURNS,VaReps)))

Being solved by Mosek...
Solution status: optimal
Running time: 0.0104s
Estimated VaR is 0.2074
VaR from 2000 to 2009 is 0.0712
VaR from 2010 to 2014 is 0.0866
VaR with extreme distribution is 0.2396


### Budgeted uncertainty set from Exercise 3.1)

$$\mathcal{Z}:= \{z\in\mathbb{R}^m\,|\,z_i\in[-1,\,1],, \|z\|_1 \leq \Gamma\}$$

In [ ]:
model = ro.Model('solveRobustPortfolio_Budget')
x=model.dvar(nStocks)

z=model.rvar(nStocks)

BudgetedSet=(z>=-1, z<=1, rso.norm(z,1)<=Gamma)
model.minmax(-(mu+P@z)@x,BudgetedSet)
model.st(x<=1)
model.st(sum(x)==1)
model.st(x>=0)
model.solve(my_solver)

obj_Budget=model.get()
xx_Budget=x.get() #get optimal portfolio

print('Estimated VaR is {0:0.4f}'.format(obj_Budget))
print('VaR from 2000 to 2009 is {0:0.4f}'.format(testPolicyVaR(xx_Budget, Rs,VaReps)))
print('VaR from 2010 to 2014 is {0:0.4f}'.format(testPolicyVaR(xx_Budget, RsTest,VaReps)))
print('VaR with extreme distribution is {0:0.4f}'.format(testPolicyVaR(xx_Budget, WCRETURNS,VaReps)))

Being solved by Mosek...
Solution status: optimal
Running time: 0.0092s
Estimated VaR is 0.1671
VaR from 2000 to 2009 is 0.0641
VaR from 2010 to 2014 is 0.0776
VaR with extreme distribution is 0.2964


### CVaR uncertainty set from Exercise 3.1)
$$\mathcal{Z}:= \left\{ z\in \mathbb{R}^m\,\middle|\, \exists \theta\in\mathbb{R}^K,\, z = \sum_{i=1}^K \theta_i \bar{z}_i,\, \theta\geq 0, \, \sum_{i=1}^K \theta_i = 1,\, \theta\leq \frac{1}{K\alpha}\right\}.$$

In [ ]:
model = ro.Model('solveRobustPortfolio_CVaR')
x=model.dvar(nStocks)

CVaR_alpha=(1/lmd.get()[0])

z=model.rvar(nStocks)
t = model.rvar(nMonths)

CVaRSet=(z == (Zs @t), t>= 0, sum(t) == 1, t <= (1/(nMonths*CVaR_alpha))) 
model.minmax(-(mu+P@z)@x,CVaRSet)
model.st(x<=1)
model.st(sum(x)==1)
model.st(x>=0)
model.solve(my_solver)

obj_CVaR=model.get()
xx_CVaR=x.get() #get optimal portfolio

print('Estimated VaR is {0:0.4f}'.format(obj_CVaR))
print('VaR from 2000 to 2009 is {0:0.4f}'.format(testPolicyVaR(xx_CVaR, Rs,VaReps)))
print('VaR from 2010 to 2014 is {0:0.4f}'.format(testPolicyVaR(xx_CVaR, RsTest,VaReps)))
print('VaR with extreme distribution is {0:0.4f}'.format(testPolicyVaR(xx_CVaR, WCRETURNS,VaReps)))

Being solved by Mosek...
Solution status: optimal
Running time: 0.0216s
Estimated VaR is -0.0431
VaR from 2000 to 2009 is 0.1681
VaR from 2010 to 2014 is 0.1483
VaR with extreme distribution is 0.3243


### CVaR uncertainty set with $\alpha = 0.05$
$$\mathcal{Z}:= \left\{ z\in \mathbb{R}^m\,\middle|\, \exists \theta\in\mathbb{R}^K,\, z = \sum_{i=1}^K \theta_i \bar{z}_i,\, \theta\geq 0, \, \sum_{i=1}^K \theta_i = 1,\, \theta\leq \frac{1}{K\alpha}\right\}.$$

In [ ]:
model = ro.Model('solveRobustPortfolio_CVaR005')
x=model.dvar(nStocks)
CVaR_alpha=0.05

z=model.rvar(nStocks)
t = model.rvar(nMonths)

CVaRSet=(z == (Zs @t), t>= 0, sum(t) == 1, t <= (1/(nMonths*CVaR_alpha))) 
model.minmax(-(mu+P@z)@x,CVaRSet)
model.st(x<=1)
model.st(sum(x)==1)
model.st(x>=0)
model.solve(my_solver)

obj_CVaR005=model.get()
xx_CVaR005=x.get() #get optimal portfolio

print('Estimated VaR is {0:0.4f}'.format(obj_CVaR005))
print('VaR from 2000 to 2009 is {0:0.4f}'.format(testPolicyVaR(xx_CVaR005, Rs,VaReps)))
print('VaR from 2010 to 2014 is {0:0.4f}'.format(testPolicyVaR(xx_CVaR005, RsTest,VaReps)))
print('VaR with extreme distribution is {0:0.4f}'.format(testPolicyVaR(xx_CVaR005, WCRETURNS,VaReps)))

Being solved by Mosek...
Solution status: optimal
Running time: 0.0239s
Estimated VaR is 0.0556
VaR from 2000 to 2009 is 0.0497
VaR from 2010 to 2014 is 0.0661
VaR with extreme distribution is 0.3223


### Budgeted uncertainty set for ambiguous chance constraint found in 3.2)

In [ ]:
model = ro.Model('solveRobustPortfolio_Budget_CC')
x=model.dvar(nStocks)

z=model.rvar(nStocks)

BudgetedSet=(z>=-1, z<=1, rso.norm(z,1)<=Gamma2)
model.minmax(-(mu+P@z)@x,BudgetedSet)
model.st(x<=1)
model.st(sum(x)==1)
model.st(x>=0)
model.solve(my_solver)

obj_GammaAmbig=model.get()
xx_GammaAmbig=x.get() #get optimal portfolio

print('Estimated VaR is {0:0.4f}'.format(obj_GammaAmbig))
print('VaR from 2000 to 2009 is {0:0.4f}'.format(testPolicyVaR(xx_GammaAmbig, Rs,VaReps)))
print('VaR from 2010 to 2014 is {0:0.4f}'.format(testPolicyVaR(xx_GammaAmbig, RsTest,VaReps)))
print('VaR with extreme distribution is {0:0.4f}'.format(testPolicyVaR(xx_GammaAmbig, WCRETURNS,VaReps)))

Being solved by Mosek...
Solution status: optimal
Running time: 0.0091s
Estimated VaR is 0.3243
VaR from 2000 to 2009 is 0.1681
VaR from 2010 to 2014 is 0.1483
VaR with extreme distribution is 0.3243


### Boxed ellipsoidal uncertainty set for ambiguous chance constraint constraint found in 3.2)

In [ ]:
model = ro.Model('solveRobustPortfolio_Ellipsoidal_CC')
x=model.dvar(nStocks)

z=model.rvar(nStocks)

EllipsoidSet=(z>=-1, z<=1, rso.norm(z,2)<=gamma2)
model.minmax(-(mu+P@z)@x,EllipsoidSet)
model.st(x<=1)
model.st(sum(x)==1)
model.st(x>=0)
model.solve(my_solver)

obj_gammaAmbig=model.get()
xx_gammaAmbig=x.get() #get optimal portfolio

print('Estimated VaR is {0:0.4f}'.format(obj_gammaAmbig))
print('VaR from 2000 to 2009 is {0:0.4f}'.format(testPolicyVaR(xx_gammaAmbig, Rs,VaReps)))
print('VaR from 2010 to 2014 is {0:0.4f}'.format(testPolicyVaR(xx_gammaAmbig, RsTest,VaReps)))
print('VaR with extreme distribution is {0:0.4f}'.format(testPolicyVaR(xx_gammaAmbig, WCRETURNS,VaReps)))

Being solved by Mosek...
Solution status: optimal
Running time: 0.0100s
Estimated VaR is 0.3243
VaR from 2000 to 2009 is 0.1681
VaR from 2010 to 2014 is 0.1483
VaR with extreme distribution is 0.3243
